In [3]:
import pandas as pd
from gensim.models import Word2Vec
from itertools import product


In [4]:


# Specify the path to your Parquet file
parquet_file_path = "C:/Users/youss/OneDrive - aucegypt.edu/Desktop/files/ML projects/intrusion detection/database.parquet"

# Read the Parquet file into a pandas DataFrame
df = pd.read_parquet(parquet_file_path)
df


,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack
0,13.58.98.64,40894,172.31.69.25,22,6,92.00,3164,23,3765,21,...,26883,26847,0,0,0,0,0,0,1,SSH-Bruteforce
1,213.202.230.143,29622,172.31.66.103,3389,6,0.00,1919,14,2031,11,...,8192,64000,0,0,0,0,0,0,0,Benign
2,172.31.66.5,65456,172.31.0.2,53,17,0.00,116,2,148,2,...,0,0,0,0,2511,1,5,0,0,Benign
3,172.31.64.92,57918,172.31.0.2,53,17,0.00,70,1,130,1,...,0,0,0,0,3371,1,60,0,0,Benign
4,18.219.32.43,63269,172.31.69.25,80,6,7.00,232,5,1136,4,...,8192,26883,0,0,0,0,0,0,1,DDoS attacks-LOIC-HTTP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18893703,172.31.66.111,59578,172.31.0.2,53,17,0.00,77,1,138,1,...,0,0,0,0,14428,1,60,0,0,Benign
18893704,172.31.67.86,52954,104.16.84.55,443,6,91.22,1729,20,6719,16,...,8192,29200,0,0,0,0,0,0,0,Benign
18893705,18.218.115.60,59664,172.31.69.28,80,6,7.00,561,5,1147,5,...,65535,26883,0,0,0,0,0,0,1,DDOS attack-HOIC
18893706,23.246.192.59,1780,172.31.66.15,445,6,41.00,120,3,124,3,...,65392,65392,0,0,0,0,0,0,0,Benign


In [5]:

# Specify the columns you want to keep
selected_columns = ['IPV4_SRC_ADDR', 'IPV4_DST_ADDR', 'L4_DST_PORT']

# Create a new DataFrame with only the selected columns
new_df = df[selected_columns]


In [6]:
new_df

,IPV4_SRC_ADDR,IPV4_DST_ADDR,L4_DST_PORT
0,13.58.98.64,172.31.69.25,22
1,213.202.230.143,172.31.66.103,3389
2,172.31.66.5,172.31.0.2,53
3,172.31.64.92,172.31.0.2,53
4,18.219.32.43,172.31.69.25,80
...,...,...,...
18893703,172.31.66.111,172.31.0.2,53
18893704,172.31.67.86,104.16.84.55,443
18893705,18.218.115.60,172.31.69.28,80
18893706,23.246.192.59,172.31.66.15,445


In [7]:
sample_size = 100000
sampled_data = new_df.head(sample_size)[["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "L4_DST_PORT"]].values.tolist()
#testing with less size 

In [8]:

# Create Skip-gram pairs with source IP as target and port/dest IP as context
skipgram_pairs = []
for entry in sampled_data:
    src_ip, dst_ip, port = entry
    # Create pairs with source IP as target and port/dest IP as context
    skipgram_pairs.extend(product([src_ip], [dst_ip, str(port)]))
    skipgram_pairs.extend(product([str(port)], [dst_ip]))
    
skipgram_pairs

[('13.58.98.64', '172.31.69.25'),
 ('13.58.98.64', '22'),
 ('22', '172.31.69.25'),
 ('213.202.230.143', '172.31.66.103'),
 ('213.202.230.143', '3389'),
 ('3389', '172.31.66.103'),
 ('172.31.66.5', '172.31.0.2'),
 ('172.31.66.5', '53'),
 ('53', '172.31.0.2'),
 ('172.31.64.92', '172.31.0.2'),
 ('172.31.64.92', '53'),
 ('53', '172.31.0.2'),
 ('18.219.32.43', '172.31.69.25'),
 ('18.219.32.43', '80'),
 ('80', '172.31.69.25'),
 ('172.31.65.93', '23.36.33.52'),
 ('172.31.65.93', '443'),
 ('443', '23.36.33.52'),
 ('172.31.68.18', '172.31.0.2'),
 ('172.31.68.18', '53'),
 ('53', '172.31.0.2'),
 ('172.31.65.118', '172.31.0.2'),
 ('172.31.65.118', '53'),
 ('53', '172.31.0.2'),
 ('18.219.32.43', '172.31.69.28'),
 ('18.219.32.43', '80'),
 ('80', '172.31.69.28'),
 ('172.31.65.93', '172.31.0.2'),
 ('172.31.65.93', '53'),
 ('53', '172.31.0.2'),
 ('213.202.230.143', '172.31.65.121'),
 ('213.202.230.143', '3389'),
 ('3389', '172.31.65.121'),
 ('172.31.64.27', '172.31.0.2'),
 ('172.31.64.27', '53'),
 ('53

In [9]:

# Train Skip-gram Word2Vec model
model_skipgram_IP = Word2Vec(sentences=skipgram_pairs, vector_size=10, window=5, sg=1, min_count=1, workers=4)

# Save or use the learned embeddings
model_skipgram_IP.save("skipgram_model_src_ip_as_target.model")

In [11]:
# Assuming model_skipgram_IP is your trained Skip-gram Word2Vec model
target_ip = "213.202.230.143"

# Check if the target_ip is in the vocabulary
if target_ip in model_skipgram_IP.wv:
    # Get the vector for the target_ip
    vector = model_skipgram_IP.wv[target_ip]
    print(f"Vector for {target_ip}: {vector}")
else:
    print(f"{target_ip} is not in the vocabulary.")

Vector for 213.202.230.143: [ 0.51958925  0.57349354 -0.73469514 -0.03171037 -1.2503922  -0.976618
  1.4747074   0.39665842 -1.1888376   0.93039834]
